In [1]:
import geopandas
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# load LA neighborhood geo data
geojson_file_loc = "data/mapping-la-data-main/geojson/la-county-neighborhoods-v6.geojson"
geo_la_neighborhood = geopandas.read_file(geojson_file_loc)
# LA EPSG transformation
geo_la_neighborhood = geo_la_neighborhood.to_crs(3310)
geo_la_neighborhood["area"] = geo_la_neighborhood.area/2.59e+6
geo_la_neighborhood = geo_la_neighborhood.to_crs(4326)
geo_la_neighborhood['boundary'] = geo_la_neighborhood.boundary
geo_la_neighborhood['centroid'] = geo_la_neighborhood.centroid

# load LA neighborhood poplulation data
pop_la_neighborhood = pd.read_csv('data/la-neighborhood-population.csv')
# merge two dataframe together
df = geo_la_neighborhood.merge(pop_la_neighborhood, left_on='name', right_on='Neighborhood', how='inner')
del df['Neighborhood']
del df['kind']
del df['external_id']
del df['slug']
del df['set']
del df['metadata']
del df['resource_uri']
df = df.rename(columns={'Population per Sqmi':'pop_density'})
df['pop'] = round(df['pop_density'] * df['area'])

C:\Users\mcran\AppData\Local\Temp\ipykernel_19532\2261507890.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geo_la_neighborhood['centroid'] = geo_la_neighborhood.centroid


In [3]:
# load LA existing vertiport data
df_heliport = pd.read_csv('data/us-heliports-landing-facilities.csv')
del df_heliport['State_Name']
del df_heliport['Facility_Type']
del df_heliport['County_s_State_Post_Office_Code']
df_airport = pd.read_csv('data/us-general-aviation-airports.csv')
del df_airport['County_s_State_Post_Office_Code']
del df_airport['State_Name']
df_vertiport = pd.concat([df_heliport, df_airport])
mask = (df_vertiport['SHAPE_X'] <-117.6) & (df_vertiport['SHAPE_X'] > -119) & (df_vertiport['SHAPE_Y'] <34.8) & (df_vertiport['SHAPE_Y'] > 33.6)
df_vertiport_la = df_vertiport[mask]

In [4]:
from scipy.sparse import csr_matrix
df_centroid = pd.DataFrame(df[['name','centroid','pop']])
n = len(df_centroid)
df_centroid['idx'] = df_centroid.index
# Cartesian product to calculate distance between all centroid pairs
centroid_pair = df_centroid.merge(df_centroid, how='cross')
gs_centroid_pair = geopandas.GeoDataFrame(centroid_pair, geometry=centroid_pair['centroid_x'], crs='EPSG:3310')
gs_centroid_pair['distance']=gs_centroid_pair.apply(lambda x: x['centroid_x'].distance(x['centroid_y']), axis=1)
gs_centroid_pair['total_pop']=gs_centroid_pair.apply(lambda x: x['pop_x'] + x['pop_y'], axis=1)
# distance matrix
D = csr_matrix((gs_centroid_pair['distance'], (gs_centroid_pair['idx_x'], gs_centroid_pair['idx_y'])), shape=(n,n))
# population matrix
# P = csr_matrix((gs_centroid_pair['total_pop'], (gs_centroid_pair['idx_x'], gs_centroid_pair['idx_y'])), shape=(n,n))
# P = P / np.max(P)

In [5]:
from sklearn.manifold import Isomap
from sklearn.cluster import KMeans
embedding = Isomap(n_neighbors=10,n_components=2)
D_transformed = embedding.fit_transform(D)

sum_of_squared_distiance=[]
K = range(1,20)
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=0).fit(D_transformed)
    sum_of_squared_distiance.append(kmeans.inertia_)

c:\Users\mcran\anaconda3\envs\geo_env\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
c:\Users\mcran\anaconda3\envs\geo_env\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
c:\Users\mcran\anaconda3\envs\geo_env\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
c:\Users\mcran\anaconda3\envs\geo_env\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarn

In [6]:
from pulp import *
import pandas as pd

# generate optimization table
new_results_df = pd.DataFrame(columns = ['Clusters','Percent Utilization','Total Profit','Number of Daily Flights','Number of eVTOLs Needed'])

for k in range(7,11):
    

        K = k
        kmeans_alg = KMeans(n_clusters=k, random_state=0).fit(D_transformed)
        df['label']=kmeans_alg.labels_

        df_pre_dissolve = geopandas.GeoDataFrame(df, geometry=df['boundary'],crs='EPSG:4326')
        df_dissolve = df[['geometry', 'pop', 'label']].dissolve(by='label', aggfunc='sum')
        df_dissolve['centroid'] = df_dissolve.centroid
        
        # calculate population ij, and distance ij
        df_dissolve = df_dissolve.to_crs(3310)
        for i in df_dissolve.index:
            df_dissolve['pop_'+ str(i)] = df_dissolve.apply(lambda x: (x['pop'] + df_dissolve['pop'][i])/K, axis=1)
            df_dissolve.loc[i, 'pop_'+ str(i)] = 0
            df_dissolve['dist_'+ str(i)] = df_dissolve.apply(lambda x: x['geometry'].distance(df_dissolve.loc[i, 'geometry'])/1609.34, axis=1)
            # set percentage of population will use eVTOL as commute 
            # total commute is 5% of population, eVTOL commute is 2.5% of total commute
            #df_dissolve['eVTOL%_'+ str(i)] = j/100

        # assign existing vertiport label
        # find which cluster center a vertiport is close to
        def min_dist(x, k=K):
            dist = []
            for i in range(k):
                dist.append(x['to_center_'+str(i)])
            if min(dist) < 0.2: # threshold can be changed
                return dist.index(min(dist))
            else:
                # ignore vertiport that is far away to any cluster center
                return 10

        ## This is what we have to come back and change... we need to be optimizing for percent of population
        def total_commute(x, k=K):
            # P=0.5 net profit per passenger per mile
            commute = 0
            for i in range(k):
                commute += x['pop_'+ str(i)] #* x['eVTOL%_'+str(i)]
            return round(commute)


        def total_prof(x, k=K, P=0.5):
            # P=0.5 net profit per passenger per mile
            prof = 0
            for i in range(k):
                prof += x['pop_'+ str(i)] * x['dist_'+ str(i)] #* x['eVTOL%_'+str(i)]
            return round(prof * P)

        gs_vertiport = geopandas.GeoSeries.from_wkt(df_vertiport_la['WKT'])
        gdf_vertiport = geopandas.GeoDataFrame(df_vertiport_la[['WKT', 'SHAPE_X', 'SHAPE_Y']], geometry=gs_vertiport, crs="EPSG:4326")
        for i in df_dissolve.index:
            gdf_vertiport['to_center_'+str(i)] = gdf_vertiport.apply(lambda x: x['geometry'].distance(df_dissolve.loc[i, 'centroid']), axis=1)
        gdf_vertiport['cluster'] = gdf_vertiport.apply(min_dist, axis=1)

        # df_dissolve['eVTOL%'] = 0.25
        df_dissolve['existing_vertiport'] = gdf_vertiport.groupby(['cluster']).size()[:-1]
        df_dissolve['total_commute'] = df_dissolve.apply(total_commute, axis=1)
        df_dissolve['total_prof'] = df_dissolve.apply(total_prof, axis=1)
        df_dissolve.to_csv('data/cluster.csv')
        # run optimization.py in pycharm




        result = pd.read_csv("data/optimization_result.csv", header=0)
        final_result_df = pd.concat([df_dissolve.iloc[:, -3:], result], axis=1)
        final_result_df.to_csv('data/result.csv')

        result = pd.read_csv("data/optimization_result.csv", header=0)
        final_result_df = pd.concat([df_dissolve.iloc[:, -3:], result], axis=1)
        final_result_df.to_csv('data/result.csv')


        data = pd.read_csv("data/cluster.csv", header=0)


        # only existing vertiport and total profit columns
        clusterName = data['label']
        dataTable = data.iloc[:, -3:].values.tolist()
        print(dataTable)

        n_Ve = dict([(i, n[0]) for i, n in enumerate(dataTable)])
        total_commute = dict([(i, n[1]) for i, n in enumerate(dataTable)])
        total_prof = dict([(i, n[2]) for i, n in enumerate(dataTable)])

        prob = LpProblem('MaxProfit', LpMinimize)

        # number of existing vertiport
        n_eVTOL_Vars = LpVariable.dicts('n_eVTOL', clusterName, 0, cat='Integer')
        # number of new vertiport
        n_Vn_Vars = LpVariable.dicts('n_Vn', clusterName, 0, cat='Integer')
        #Percent_utilization
        util_percent_Vars = LpVariable.dicts('util_percent', clusterName, lowBound=0, upBound=0.3, cat='Continuous')

        n_round = 15
        c_eVTOL = 1000000.0 ## updated to 1 million
        c_Ve = 200000.0
        c_Vn = 500000.0
        n_Vt = 50
        cap_eVTOL = 4.0
        cap_port = 10.0
        depreciation_1 = 1/365
        depreciation_10 = 1/365/10
        depreciation_20 = 1/365/20
        total_capital = 5000000000 #1,000,000,000

        # Objection function
        prob += lpSum([- total_prof[i]*util_percent_Vars[i] + c_eVTOL * n_eVTOL_Vars[i]*depreciation_10 + c_Ve * n_Ve[i]* util_percent_Vars[i]*depreciation_1 + c_Vn *
               n_Vn_Vars[i] * depreciation_20 for i in clusterName]), 'Total Profit'

        # Constraint
        for i in clusterName:
            prob += cap_eVTOL * n_eVTOL_Vars[i] * n_round >= total_commute[i]*util_percent_Vars[i]
            prob += cap_port* (n_Ve[i] + n_Vn_Vars[i]) >= n_eVTOL_Vars[i]
            prob += n_Vn_Vars[i] +  n_Ve[i] <= n_Vt
            prob += n_Vn_Vars[i]*c_Vn + n_Ve[i]*c_Vn + n_eVTOL_Vars[i]*c_eVTOL<= total_capital
        
        prob.solve()

        #print()

        for var in prob.variables():
            print(str(var) + ' : ' + str(var.varValue))
        print()

        print("Total profit = $%.2f" % value(-prob.objective))
        # res_dict = {'new_vertiport': [var.varValue[:10] for var in prob.variables()],
        #             'eVTOL#': [var.varValue[10:] for var in prob.variables()]}
        # res = pd.DataFrame(res_dict)
        res = [var.varValue for var in prob.variables()]
        res_df = pd.DataFrame(columns=['new_vertiport', 'eVTOL#'])
        res_df['new_vertiport'] = res[:K]
        res_df['eVTOL#'] = res[K:2*K]
        res_df['util_precent'] = res[2*K:]
        #res_df.to_csv('data/optimization_result.csv', index=False)`
        #['Clusters','Percent Utilization','Total Profit','Number of Daily Flights','Number of eVTOLs Needed']
        results_row = pd.DataFrame([k,sum(res[2*K:]),value(-prob.objective),n_round*sum(res[K:]),sum(res[K:]) ]).T
        results_row.columns =  ['Clusters','Percent Utilization','Total Profit','Number of Daily Flights','Number of eVTOLs Needed']
    
        new_results_df = pd.concat([new_results_df,results_row])
new_results_df

c:\Users\mcran\anaconda3\envs\geo_env\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\mcran\AppData\Local\Temp\ipykernel_19532\82915327.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_dissolve['centroid'] = df_dissolve.centroid


[[18, 2974269, 5211994], [21, 2232082, 1354394], [6, 1568281, 8691243], [46, 3349972, 9716819], [9, 1527381, 6612635], [9, 2067971, 2277268], [39, 2510183, 2582215]]
n_Vn_0 : 0
n_Vn_1 : 0
n_Vn_2 : 44
n_Vn_3 : 0
n_Vn_4 : 0
n_Vn_5 : 0
n_Vn_6 : 0
n_eVTOL_0 : 0
n_eVTOL_1 : 0
n_eVTOL_2 : 500
n_eVTOL_3 : 0
n_eVTOL_4 : 0
n_eVTOL_5 : 0
n_eVTOL_6 : 0
util_percent_0 : 0.0
util_percent_1 : 0.0
util_percent_2 : 0.0191292
util_percent_3 : 0.0
util_percent_4 : 0.0
util_percent_5 : 0.0
util_percent_6 : 0.0

Total profit = $26193.64


c:\Users\mcran\anaconda3\envs\geo_env\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\mcran\AppData\Local\Temp\ipykernel_19532\82915327.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_dissolve['centroid'] = df_dissolve.centroid


[[17, 1563149, 7259064], [17, 2831767, 6638387], [52, 2434125, 4195994], [2, 1397972, 8237782], [9, 1894883, 2454793], [32, 3099251, 10615712], [3, 1217477, 6653469], [16, 2129642, 479332]]
n_Vn_0 : 0
n_Vn_1 : 0
n_Vn_2 : 0
n_Vn_3 : 0
n_Vn_4 : 0
n_Vn_5 : 0
n_Vn_6 : 0
n_Vn_7 : 0
n_eVTOL_0 : 0
n_eVTOL_1 : 0
n_eVTOL_2 : 0
n_eVTOL_3 : 0
n_eVTOL_4 : 0
n_eVTOL_5 : 0
n_eVTOL_6 : 0
n_eVTOL_7 : 0
util_percent_0 : 0.0
util_percent_1 : 0.0
util_percent_2 : 0.0
util_percent_3 : 0.0
util_percent_4 : 0.0
util_percent_5 : 0.0
util_percent_6 : 0.0
util_percent_7 : 0.0

Total profit = $0.00


c:\Users\mcran\anaconda3\envs\geo_env\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\mcran\AppData\Local\Temp\ipykernel_19532\82915327.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_dissolve['centroid'] = df_dissolve.centroid


[[17, 1445718, 7550890], [16, 2710965, 5746840], [52, 2348952, 4837044], [2, 1274424, 7927523], [3, 1347959, 1714751], [32, 3038713, 10161622], [3, 1087243, 6588608], [16, 2033192, 1395183], [7, 1544090, 5800902]]
n_Vn_0 : 0
n_Vn_1 : 0
n_Vn_2 : 0
n_Vn_3 : 0
n_Vn_4 : 0
n_Vn_5 : 0
n_Vn_6 : 0
n_Vn_7 : 0
n_Vn_8 : 0
n_eVTOL_0 : 0
n_eVTOL_1 : 0
n_eVTOL_2 : 0
n_eVTOL_3 : 0
n_eVTOL_4 : 0
n_eVTOL_5 : 0
n_eVTOL_6 : 0
n_eVTOL_7 : 0
n_eVTOL_8 : 0
util_percent_0 : 0.0
util_percent_1 : 0.0
util_percent_2 : 0.0
util_percent_3 : 0.0
util_percent_4 : 0.0
util_percent_5 : 0.0
util_percent_6 : 0.0
util_percent_7 : 0.0
util_percent_8 : 0.0

Total profit = $0.00


c:\Users\mcran\anaconda3\envs\geo_env\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\Users\mcran\AppData\Local\Temp\ipykernel_19532\82915327.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_dissolve['centroid'] = df_dissolve.centroid


[[22, 1918295, 5361403], [4, 1558239, 2625041], [18, 2394982, 9606478], [3, 968232, 7357614], [14, 1840728, 6691426], [12, 1853219, 3780370], [19, 1212745, 7168309], [5, 1175469, 8620766], [13, 2340289, 9151388], [49, 1779447, 2433946]]
n_Vn_0 : 0
n_Vn_1 : 0
n_Vn_2 : 0
n_Vn_3 : 47
n_Vn_4 : 0
n_Vn_5 : 0
n_Vn_6 : 31
n_Vn_7 : 45
n_Vn_8 : 0
n_Vn_9 : 0
n_eVTOL_0 : 0
n_eVTOL_1 : 0
n_eVTOL_2 : 0
n_eVTOL_3 : 500
n_eVTOL_4 : 0
n_eVTOL_5 : 0
n_eVTOL_6 : 500
n_eVTOL_7 : 500
n_eVTOL_8 : 0
n_eVTOL_9 : 0
util_percent_0 : 0.0
util_percent_1 : 0.0
util_percent_2 : 0.0
util_percent_3 : 0.0309843
util_percent_4 : 0.0
util_percent_5 : 0.0
util_percent_6 : 0.0247373
util_percent_7 : 0.0255217
util_percent_8 : 0.0
util_percent_9 : 0.0

Total profit = $205549.78


,Clusters,Percent Utilization,Total Profit,Number of Daily Flights,Number of eVTOLs Needed
0,7.0,0.019129,26193.635075,7500.286938,500.019129
0,8.0,0.000000,0.000000,0.000000,0.000000
0,9.0,0.000000,0.000000,0.000000,0.000000
0,10.0,0.081243,205549.777089,22501.218649,1500.081243


In [7]:
from pulp import *
import pandas as pd

data = pd.read_csv("data/cluster.csv", header=0)


# only existing vertiport and total profit columns
clusterName = data['label']
dataTable = data.iloc[:, -3:].values.tolist()
print(dataTable)

n_Ve = dict([(i, n[0]) for i, n in enumerate(dataTable)])
total_commute = dict([(i, n[1]) for i, n in enumerate(dataTable)])
total_prof = dict([(i, n[2]) for i, n in enumerate(dataTable)])

prob = LpProblem('MaxProfit', LpMinimize)

# number of existing vertiport
n_eVTOL_Vars = LpVariable.dicts('n_eVTOL', clusterName, 0, cat='Integer')
# number of new vertiport
n_Vn_Vars = LpVariable.dicts('n_Vn', clusterName, 0, cat='Integer')

n_round = 15
c_eVTOL = 1000000.0 ## updated to 1 million
c_Ve = 200000.0
c_Vn = 500000.0
cap_eVTOL = 4.0
cap_port = 8.0
depreciation_1 = 1/365
depreciation_10 = 1/365/10
depreciation_20 = 1/365/20

# Objection function
prob += lpSum([- total_prof[i] + c_eVTOL * n_eVTOL_Vars[i]*depreciation_10 + c_Ve * n_Ve[i]*depreciation_1 + c_Vn *
               n_Vn_Vars[i] * depreciation_20 for i in clusterName]), 'Total Profit'

# Constraint
for i in clusterName:
    prob += cap_eVTOL * n_eVTOL_Vars[i] * n_round >= total_commute[i]
    prob += cap_port * (n_Ve[i] + n_Vn_Vars[i]) >= n_eVTOL_Vars[i]

prob.solve()

print()

for var in prob.variables():
    print(str(var) + ' : ' + str(var.varValue))
print()

print("Total profit = $%.2f" % value(-prob.objective))
# res_dict = {'new_vertiport': [var.varValue[:10] for var in prob.variables()],
#             'eVTOL#': [var.varValue[10:] for var in prob.variables()]}
# res = pd.DataFrame(res_dict)
res = [var.varValue for var in prob.variables()]
res_df = pd.DataFrame(columns=['new_vertiport', 'eVTOL#'])
res_df['new_vertiport'] = res[:10]
res_df['eVTOL#'] = res[10:]
#res_df.to_csv('data/optimization_result.csv', index=False)

[[22, 1918295, 5361403], [4, 1558239, 2625041], [18, 2394982, 9606478], [3, 968232, 7357614], [14, 1840728, 6691426], [12, 1853219, 3780370], [19, 1212745, 7168309], [5, 1175469, 8620766], [13, 2340289, 9151388], [49, 1779447, 2433946]]

n_Vn_0 : 3975
n_Vn_1 : 3243
n_Vn_2 : 4972
n_Vn_3 : 2015
n_Vn_4 : 3821
n_Vn_5 : 3849
n_Vn_6 : 2508
n_Vn_7 : 2444
n_Vn_8 : 4863
n_Vn_9 : 3659
n_eVTOL_0 : 31972
n_eVTOL_1 : 25971
n_eVTOL_2 : 39917
n_eVTOL_3 : 16138
n_eVTOL_4 : 30679
n_eVTOL_5 : 30887
n_eVTOL_6 : 20213
n_eVTOL_7 : 19592
n_eVTOL_8 : 39005
n_eVTOL_9 : 29658

Total profit = $-17528532.97


In [8]:
data

,label,geometry,pop,centroid,pop_0,dist_0,pop_1,dist_1,pop_2,dist_2,...,dist_6,pop_7,dist_7,pop_8,dist_8,pop_9,dist_9,existing_vertiport,total_commute,total_prof
0,0,MULTILINESTRING ((149771.67112347204 -442445.4...,1214421.0,POINT (-118.4017715911054 34.06434931240423),0.0,0.000000,197877.2,5.466977,302470.1,5.361932,...,3.967058,150031.0,21.016524,295633.5,0.000000,225528.2,0.000000,22,1918295,5361403
1,1,MULTILINESTRING ((174772.41958729725 -391879.5...,764351.0,POINT (-117.89807531601825 34.137793434079526),197877.2,5.466977,0.0,0.000000,257463.1,1.058209,...,14.104341,105024.0,0.000000,250626.5,9.371829,180521.2,0.000000,4,1558239,2625041
2,2,MULTILINESTRING ((172154.6478548056 -454573.40...,1810280.0,POINT (-118.19473570919335 33.87598418360855),302470.1,5.361932,257463.1,1.058209,0.0,0.000000,...,19.841771,209616.9,22.664749,355219.4,0.000000,285114.1,0.000000,18,2394982,9606478
3,3,MULTILINESTRING ((164502.0206796736 -384804.64...,26843.0,POINT (-118.43708102592144 34.62362212208346),124126.4,14.914783,79119.4,0.000000,183712.3,26.318615,...,0.000000,31273.2,0.000000,176875.7,25.003240,106770.4,13.167457,3,968232,7357614
4,4,MULTILINESTRING ((182769.80986845182 -424600.2...,1117462.0,POINT (-118.0295766456479 34.02570119732147),233188.3,10.296730,188181.3,0.000000,292774.2,0.000000,...,23.495960,140335.1,12.847751,285937.6,6.168659,215832.3,0.000000,14,1840728,6691426
5,5,MULTILINESTRING ((143396.05630828597 -415684.7...,1133076.0,POINT (-118.43091491784193 34.12777392511734),234749.7,0.000000,189742.7,0.000000,294335.6,11.826945,...,0.000000,141896.5,0.000000,287499.0,7.902637,217393.7,0.000000,12,1853219,3780370
6,6,MULTILINESTRING ((112815.63015896079 -426784.7...,332484.0,POINT (-118.64326141611919 34.26762080145649),154690.5,3.967058,109683.5,14.104341,214276.4,19.841771,...,0.000000,61837.3,8.849012,207439.8,11.570976,137334.5,11.432287,19,1212745,7168309
7,7,MULTILINESTRING ((155370.35713770363 -516170.5...,285889.0,POINT (-118.02244317128229 34.5725295741208),150031.0,21.016524,105024.0,0.000000,209616.9,22.664749,...,8.849012,0.0,0.000000,202780.3,25.127433,132675.0,14.260036,5,1175469,8620766
8,8,MULTILINESTRING ((156232.58503560742 -440624.4...,1741914.0,POINT (-118.32075059793034 33.91219003709535),295633.5,0.000000,250626.5,9.371829,355219.4,0.000000,...,11.570976,202780.3,25.127433,0.0,0.000000,278277.5,0.000000,13,2340289,9151388
9,9,MULTILINESTRING ((173279.84962499834 -432798.4...,1040861.0,POINT (-118.20039910815179 34.132871192047006),225528.2,0.000000,180521.2,0.000000,285114.1,0.000000,...,11.432287,132675.0,14.260036,278277.5,0.000000,0.0,0.000000,49,1779447,2433946
